In [1]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/03 21:52:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
spark.version

'3.3.2'

In [3]:
df = spark.read \
    .option("inferSchema", "true") \
    .option("header", "true") \
    .csv(f'data/raw/')

In [7]:
df.repartition(12) \
    .write.parquet(f'data/pq/')

23/03/03 22:04:53 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [8]:
df = spark.read.parquet('data/pq/')

In [9]:
df.createOrReplaceTempView('trips_data')

In [44]:
spark.sql("""
    select
        COUNT(*)
    from
       trips_data
    where 
        DATE(pickup_datetime) = TO_DATE('2021.06.15','yyyy.MM.dd')
""").show()

+--------+
|count(1)|
+--------+
|  452470|
+--------+



In [66]:
spark.sql("""
    select
        MAX(ROUND(DATEDIFF(minute, pickup_datetime, dropoff_datetime)/60, 2)) Hours
    from
       trips_data
""").show()

+-----+
|Hours|
+-----+
|66.87|
+-----+



In [67]:
df_zones = spark.read\
    .option("inferSchema", "true") \
    .option("header", "true") \
    .csv('data/zones/')

In [71]:
df_zones.createOrReplaceTempView('zones_data')

In [73]:
df_zones.select(df_zones.columns).show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [74]:
df.columns

['dispatching_base_num',
 'pickup_datetime',
 'dropoff_datetime',
 'PULocationID',
 'DOLocationID',
 'SR_Flag',
 'Affiliated_base_number']

In [77]:
spark.sql("""
    select
        COUNT(*) count_trips,
        zones.Zone
    from
        trips_data trips 
            left join zones_data zones on trips.PULocationID = zones.LocationID
    group by
        zones.Zone
    order by
        count_trips desc
    limit 1
""").show()

+-----------+-------------------+
|count_trips|               Zone|
+-----------+-------------------+
|     231279|Crown Heights North|
+-----------+-------------------+

